# EPL 데이터 분석

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

from matplotlib.pylab import rcParams, style
rcParams['figure.figsize'] = 12, 8
rcParams['font.family'] = 'New Gulim'
rcParams['font.size'] = 10

style.use('ggplot')

## 데이터 로드

In [ ]:
# 2015-2016 잉글리시 프리미어 리그 경기기록
df = pd.read_excel('data/epl.xls')
df.head()

In [ ]:
df['HomeTeam'].unique() # array

In [ ]:
df['HomeTeam'].value_counts()

In [ ]:
df['Date']

In [ ]:
# H / A 중복되어 들어 있나? 2015년 8월 8일 경기 기록 확인.
df[df['Date'] == '2015-08-08']

In [ ]:
# FTHG = Full Time Home Team Goal
# FTAG = Full Time Away Team Goal
# FTR  = Full Time Result (H / A / D) : ( Home / Awaw / Draw )

df_w = df[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']]
df_w.head()

##  1. 시즌 성적표 작성

In [ ]:
teams = df['HomeTeam'].unique()
teams

In [ ]:
# 시즌 성적표(빈 양식)

# Point(승점) = W * 3 + D

f_tab = pd.DataFrame({'Team': teams, 'Point': 0},
                     columns={'Team', 'Win', 'Draw', 'Lost', '득점', '실점', '골득실', 'Point'})
f_tab = f_tab.fillna(0)
f_tab.head()

In [ ]:
f_tab = f_tab.reindex(columns= ['Team', 'Win', 'Draw', 'Lost', '득점', '실점', '골득실', 'Point'])
f_tab.head()

#### 데이터 탐색

In [ ]:
df_w.head()

In [ ]:
# HomeTeam으로 그룹핑
gp_home = df_w.groupby('HomeTeam')

In [ ]:
#for name, group in gp_home:
#    print(name)
#    print(group)

dic_t = dict(list(gp_home))

In [ ]:
dic_t['Liverpool']

In [ ]:
dic_t['Chelsea']

In [ ]:
# 각 팀별로 득점, 실점, 골득실을 계산

f_tab.loc[
    f_tab['Team'] == 'Arsenal', '득점'
] = dic_t['Arsenal']['FTHG'].sum()


f_tab.loc[
    f_tab['Team'] == 'Arsenal', '실점'
] = dic_t['Arsenal']['FTAG'].sum()

f_tab['골득실'] = f_tab['득점'] - f_tab['실점']

f_tab

In [ ]:
# 각 그룹의 첫 번째 데이터
gp_home.first()

In [ ]:
# 각 그룹의 마지막 데이터
gp_home.last()

#### Home team

In [ ]:
# 각 팀별로 득점, 실점 계산
df_home = gp_home.agg({'FTHG': sum, 'FTAG': sum })
df_home.rename(columns={'FTHG': '득점', 'FTAG': '실점'}, inplace=True)
df_home

In [ ]:
# 함수정의: 승, 패, 무승부 횟수 계산
def func(x):
    return x.value_counts()


In [ ]:
# 각 팀별로 승, 패, 무승부 횟수 계산
df_home_ftr = gp_home['FTR'].apply(func).unstack()
df_home_ftr

In [ ]:
# 컬럼 이름 수정 ( A: Away team 승, D: 무승부, H: Home team 승 )
df_home_ftr.rename(columns={'A': 'Lost', 'D': 'Draw', 'H': 'Win'}, inplace=True)
df_home_ftr

#### Away team

In [ ]:
# HomeTeam으로 그룹핑
gp_away = df_w.groupby('AwayTeam')

In [ ]:
# 각 팀별로 득점, 실점 계산
df_away = gp_away.agg({'FTHG': sum, 'FTAG': sum })
df_away.rename(columns={'FTHG': '실점', 'FTAG': '득점'}, inplace=True)
df_away

In [ ]:
# 각 팀별로 승, 패, 무승부 횟수 계산
df_away_ftr = gp_away['FTR'].apply(func).unstack()
df_away_ftr

In [ ]:
# 컬럼 이름 수정 ( A: Away team 승, D: 무승부, H: Home team 승 )
df_away_ftr.rename(columns={'A': 'Win', 'D': 'Draw', 'H': 'Lost'}, inplace=True)
df_away_ftr

#### Home + Away: 득점, 실점, 골득실 계산

In [ ]:
# 홈/원정 경기에 대한 득점/실점 값 합산
df_goal = df_home + df_away
df_goal.index.name = None
df_goal

In [ ]:
# 골득실 계산
df_goal['골득실'] = df_goal['득점'] - df_goal['실점']
df_goal

#### Home + Away: 승, 패, 승점 계산

In [ ]:
df_ftr = df_home_ftr + df_away_ftr
df_ftr.index.name = None
df_ftr

In [ ]:
# 승점 계산 ( 승점 = 승 * 3 + 무승부 )
df_ftr['Point'] = df_ftr['Win'] * 3 + df_ftr['Draw']
df_ftr.sort_values('Point', ascending=False)
df_ftr

#### 최종 시즌 성적표

In [ ]:
# 승/패, 골득실 Dataframe 통합
f_tab = pd.concat([df_ftr, df_goal], axis=1)

# 컬럼 순서 정리
f_tab = f_tab[['Win', 'Draw', 'Lost', '득점', '실점', '골득실', 'Point']]

# 승점 우선 정렬, 동률인 경우 골득실로 정렬
f_tab.sort_values(['Point', '골득실'], ascending=False)

---

## 2. 각 팀별 성적 집계

In [ ]:
df_w.head()

In [ ]:
# 팀 이름 설정
team_name = 'Liverpool'

In [ ]:
# Home team 일 경우의 데이터
df_w[df_w['HomeTeam'] == team_name]

In [ ]:
# Away team 일 경우의 데이터
df_w[df_w['AwayTeam'] == team_name]

#### 승, 패 처리 함수 정의

In [ ]:
def result(x, home):
    if x == 'H':
        if home:  return 'W'
        else:     return 'L'
    elif x == 'A':
        if home:  return 'L'
        else:     return 'W'
    else:
        return x
    

#### Home team

In [ ]:
df_home = df_w[df_w['HomeTeam'] == team_name]
df_home.head()

In [ ]:
# 컬럼 이름 수정
df_home = df_home.rename(columns={'AwayTeam':'Opponent', 'FTHG':'득점', 'FTAG':'실점' })
df_home.head()

In [ ]:
# 승, 패 처리
df_home['FTR'] = df_home['FTR'].apply(result, home=True)
df_home.head()

#### Away team

In [ ]:
df_away = df_w[df_w['AwayTeam'] == team_name]
df_away.head()

In [ ]:
# 컬럼 이름 수정
df_away = df_away.rename(columns={'HomeTeam':'Opponent', 'FTHG':'실점', 'FTAG':'득점' })
df_away.head()

In [ ]:
# 승, 패 처리
df_away['FTR'] = df_away['FTR'].apply(result, home=False)
df_away.head()

#### Home + Away

In [ ]:
# Merge ( Home + Away )
df_team = pd.merge(df_home, df_away, how='outer')
df_team

In [ ]:
# 컬럼 삭제
df_team.drop(['HomeTeam', 'AwayTeam'], axis=1, inplace=True)
df_team.head()

In [ ]:
# Date 타입 변경 ( str --> datetime )
df_team['Date'] = pd.to_datetime(df_team['Date'], format="%Y-%m-%d")

# Date 컬럼으로 정렬
df_team.sort_values('Date', inplace=True)

df_team.head()

In [ ]:
# Date 컬럼을 인덱스로 지정
df_team.set_index('Date', inplace=True)
df_team.head()

#### Win, CumWin 컬럼 생성

In [ ]:
# Win 컬럼 생성 ( 승: 1, 패: 0, 무승부: 0 )
df_team['Win'] = df_team['FTR'].map({'W':1, 'L':0, 'D': 0})
df_team.head()

In [ ]:
# CumWin 컬럼 생성 ( 승리 누적 합계 )
df_team['CumWin'] = df_team['Win'].cumsum()
df_team.head(5)

In [ ]:
# 누적 승리 그래프
df_team['CumWin'].plot()

#### Lost, CumLost 컬럼 생성

In [ ]:
# Lost 컬럼 생성 ( 승: 0, 패: 1, 무승부: 0 )
df_team['Lost'] = df_team['FTR'].map({'W':0, 'L':1, 'D': 0})

In [ ]:
# CumLost 컬럼 생성 ( 패배 누적 합계 )
df_team['CumLost'] = df_team['Lost'].cumsum()

In [ ]:
# 누적 패배 그래프
df_team['CumLost'].plot()

In [ ]:
# 누적 승/패 그래프
df_team[['CumWin', 'CumLost']].plot()

#### Point, CumPoint 컬럼 생성

In [ ]:
# Point 컬럼 생성 ( 승점 계산: W * 3 + D )
df_team['Point'] = df_team['FTR'].map({'W':3, 'L':0, 'D': 1})

In [ ]:
# CumPoint 컬럼 생성 ( 승점 누적 합계 )
df_team['CumPoint'] = df_team['Point'].cumsum()

In [ ]:
# 누적 승점 그래프
df_team['CumPoint'].plot()

---

##  3. 각 팀별 성적 집계 - 함수 이용

#### 성적 집계 함수 정의

In [ ]:
def team_analysis(df, team_name):

    df_w = df
    
    #----------------------------------------------------------------------------
    # 입력된 team 데이터 로드
    #----------------------------------------------------------------------------
    # Home, Away team 로드
    df_home = df_w[df_w['HomeTeam'] == team_name]
    df_away = df_w[df_w['AwayTeam'] == team_name]

    # 승, 패 처리 함수 정의
    def result(x, home):
        if x == 'H':
            if home: return 'W'
            else:    return 'L'
        elif x == 'A':
            if home: return 'L'
            else:    return 'W'
        else:
            return x
    
    #----------------------------------------------------------------------------
    # team 별 데이터 처리
    #----------------------------------------------------------------------------
    # 컬럼 이름 수정
    df_home = df_home.rename(columns={'AwayTeam':'Opponent', 'FTHG':'득점', 'FTAG':'실점' })
    df_away = df_away.rename(columns={'HomeTeam':'Opponent', 'FTHG':'실점', 'FTAG':'득점' })
    
    # 승, 패 처리
    df_home['FTR'] = df_home['FTR'].apply(result, home=True)
    df_away['FTR'] = df_away['FTR'].apply(result, home=False)


    #----------------------------------------------------------------------------
    # 데이터 통합처리 ( Home + Away )
    #----------------------------------------------------------------------------
    # Merge ( Home + Away )
    df_team = pd.merge(df_home, df_away, how='outer')
    
    # 컬럼 삭제
    df_team.drop(['HomeTeam', 'AwayTeam'], axis=1, inplace=True)

    # Date 타입 변경 ( str --> datetime )
    df_team['Date'] = pd.to_datetime(df_team['Date'], format="%Y-%m-%d")

    # 날짜별 정렬
    df_team.sort_values('Date', inplace=True)
    
    # 날짜로 인덱스 설정
    df_team.set_index('Date', inplace=True)
    

    #----------------------------------------------------------------------------
    # 집계 컬럼 생성
    #----------------------------------------------------------------------------
    # Win, CumWin 컬럼 생성
    df_team['Win'] = df_team['FTR'].map({'W':1, 'L':0, 'D': 0})
    df_team['CumWin'] = df_team['Win'].cumsum()

    # Lost, CumLost 컬럼 생성
    df_team['Lost'] = df_team['FTR'].map({'W':0, 'L':1, 'D': 0})
    df_team['CumLost'] = df_team['Lost'].cumsum()

    # Point, CumPoint 컬럼 생성
    df_team['Point'] = df_team['FTR'].map({'W':3, 'L':0, 'D': 1})
    df_team['CumPoint'] = df_team['Point'].cumsum()
    
    df_team.name = team_name
    
    return df_team

#### 각 팀별 성적 집계

In [ ]:
# 레스터 시티
leicester = team_analysis(df_w, 'Leicester')
leicester

In [ ]:
leicester['CumPoint'].plot()

In [ ]:
# 첼시
chelsea = team_analysis(df_w, 'Chelsea')
chelsea.index.name = 'Date'
chelsea.head()

In [ ]:
#import openpyxl

In [ ]:
# 파일 저장
chelsea.to_excel('data/chelsea.xlsx')

#### 여러 팀 성적 집계

In [ ]:
# 처리할 팀 리스트
teams = ['Chelsea', 'Liverpool', 'Arsenal', 'Leicester']

In [ ]:
plt.figure(figsize=(8,4))

for t in teams:
    # 누적 승점 그래프
    df = team_analysis(df_w, t)['CumPoint']
    df.plot(label=t)


plt.legend(loc='best')
plt.ylabel('Cumulative Points')

---

In [ ]:
# end of file